In [80]:
from vecstack import StackingTransformer
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor , AdaBoostRegressor
from sklearn.pipeline import Pipeline
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.externals import joblib
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error , mean_squared_log_error , r2_score
from math import sqrt
import pandas as pd

In [57]:
df = pd.read_csv('Admission_Predict_new.csv')
df.drop('Serial No.' , axis = 1 , inplace = True)

In [58]:
X = df.iloc[: , :7].values
y = df.loc[: ,'Chance of Admit '].values



In [59]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.2 , random_state = 42)

In [175]:
l0_models = [('rg'  ,Ridge(random_state = 42)) , 
            ('rf' , RandomForestRegressor(n_estimators = 100  ,random_state = 42)),
             ('ada' , AdaBoostRegressor(n_estimators = 100  ,random_state = 42)),
             ('svr' , SVR(gamma = 'auto' , kernel = 'rbf' )),
             ('xg' , XGBRegressor(n_estimators = 100 , learning_rate = 0.001 , max_depth = 3 , booster = 'gblinear' , random_state = 42))
             
            ]

#l1_models  = [#('cat' ,CatBoostRegressor(logging_level = 'Silent' , n_estimators = 100 , random_state = 42 ,learning_rate = 0.01)) ,
 #             ('xg' , XGBRegressor(n_estimators = 100 , learning_rate = 0.01 , max_depth = 4 , booster = 'gbtree' , random_state = 42))
  #           ]

f_model = LGBMRegressor(n_estimators = 100  , random_state = 40 , learning_rate = 0.1)


In [177]:
stack_1= StackingTransformer(estimators = l0_models ,regression = True, variant = 'A', n_folds = 8 , metric = mean_squared_log_error,
       
                             verbose = 0 , shuffle = True)

#stack_2 = StackingTransformer(estimators = l1_models ,regression = True, variant = 'A'  , n_folds = 10 ,
       
 #                            verbose = 0 , shuffle = True)

In [178]:
steps = [('stack_1' , stack_1),
         #('stack_2' , stack_2),
        ('final_model' , f_model)]

stack_model = Pipeline(steps)


In [179]:
stack_model = stack_model.fit(X_train , y_train)
pred = stack_model.predict(X_test)

In [180]:
mse = mean_squared_error(y_test , pred)
mse_log = mean_squared_log_error(y_test , pred)
rmse = sqrt(mse)
model_r2 = r2_score(y_test , pred)

In [181]:
print("******{}******".format('StackNet'))
print("---------------------------")
print("MSE Score : {}".format(mse))
print("RMSE : {}".format(rmse))
print("Mean Squared Log Error : {}".format(mse_log))
print("R2-Score : {}".format(model_r2))

******StackNet******
---------------------------
MSE Score : 0.003994822793393607
RMSE : 0.06320461053905488
Mean Squared Log Error : 0.0015231489007274347
R2-Score : 0.804654142132342


In [192]:
stack_result = pd.DataFrame(columns = ['MSE' , 'RMSE' , 'MSE_LOG' , 'R2-Score'])

stack_result.loc['Stack_model'] = [mse , rmse , mse_log , model_r2]

In [194]:
stack_result.to_csv('Stack_model_results.csv')

In [196]:
_ =  joblib.dump(stack_model , 'Stack_Model.pkl') #saving the model